# JPEG encoding tutorial
This jupyter notebook should give you a brief introduction to jpeg encoding from RGB image data.
The focus will be on the general concept. For details of the entire standard, please refer to: [ISO/IEC IS 10918-1](https://www.iso.org/standard/18902.html) (not free) or [ITU-T T.81](http://www.w3.org/Graphics/JPEG/itu-t81.pdf). Excellent documentation of the jpeg image format can also be found on [wikipedia](https://en.wikipedia.org/wiki/JPEG). Reference implementations and lists of publications are given on [jpeg.org](https://jpeg.org). 

### Remark on efficiency:
This code uses `numpy` to store data. However, to illustrate the algorithms used as simply as possible, no `numpy` methods will be used that act on arrays. For array data manipulation, `for` loops will be used, to allows an easy understanding of the algorithmic complexity and translation to language like *C* or *C++*. This will cost efficiency in *python*, but efficiency is not the goal of this notebook.   

In [ ]:
# load modules
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

## Load image into python
In order to work with a real image and not with random data, we load a real image (in this case a PNG) using the PIL library. 

In [ ]:
# download example files (image is under public domain and only has RGB color channels no alpha channel)
!wget -c https://upload.wikimedia.org/wikipedia/commons/thumb/e/e5/HZDR-LOGO.png/1280px-HZDR-LOGO.png -O example.png

In [ ]:
# load image
image = Image.open('example.png')

In [ ]:
# plot image
image

The image consists of 3 color channels: **R**ed **G**reen and **B**lue (RGB). This specific png does not contain transparency (alpha) values. If it would contain a fourth/alpha channel, we should convert the data beforehand using e.g. [imagemagick](https://imagemagick.org)'s `convert` tool. 

In [ ]:
# print information about the image
print("Format:", image.format)
print("Size:", image.size)
print("Color space:", image.mode)

## Convert image into numpy array and adjust its size

For easier data handling, we will use numpy arrays to deal with the pixels and the 3 color channels.

In [ ]:
# convert image to numpy array
data_RGB = np.asarray(image)

In [ ]:

print("shape:", np.shape(data_RGB))
print("max:", np.amax(data_RGB))
print("data type:", data_RGB.dtype)

The Fourie decomposition used in the `jpeg` file format requires the image to consist of $8\times8$ pixels. This is not the case for this image thus we will clip the image slightly. In a full-featured `jpeg` image this would be handled differently, but here we can rely on the image to consist of $8 \times 8$ pixels and ignore the corner cases. 

Furthermore, since we will later use downsampling on the chroma channels, the required block splitting in $\mathrm{RGB}$ and $\mathrm{Y}$ space becomes larger by a factor $2$.

Thus for simplicity, we will clip the image so that it consists of $16 \times 16$ blocks.

In [ ]:
N_y = np.shape(data_RGB)[0]
N_x = np.shape(data_RGB)[1]

for name, N in [["x", N_x], ["y", N_y]]:
    if N % 16 == 0:
        print("dimension", name, "fits into 16 pixel blocks.", "(total size: {})".format(N))
    else:
        print("dimension", name, "does not fits into 16 pixel blocks by {} pixels.".format(N % 16), "(total size: {})".format(N))
        

In [ ]:
data_RGB = data_RGB[0:(N_y // 16) * 16, 0:(N_x // 16) * 16, :]
print("new shape:", np.shape(data_RGB))

# overwrite with new shape:
N_y = np.shape(data_RGB)[0]
N_x = np.shape(data_RGB)[1]

## Convert color space from $\mathrm{RGP}$ to $\mathrm{Y C_B C_R}$

One trick used by the `jpeg` file format to reduce the size is to convert the color space in which the image is encoded. This per se does not reduce the size needed to store an image. But in contrast to [$\mathrm{RGB}$](https://en.wikipedia.org/wiki/RGB_color_model), where each color channel is equally important in human perception, in [$\mathrm{Y C_B C_R}$](https://en.wikipedia.org/wiki/YCbCr), the $\mathrm{Y}$ (["luma"](https://en.wikipedia.org/wiki/Luma_(video))) channel is most important for human perception, allowing to reduce the spatial resolution of both ["chroma"](https://en.wikipedia.org/wiki/Chrominance) channels the $\mathrm{C_B}$  ("blue-difference") and $\mathrm{C_R}$ ("red-difference") channels.  

In [ ]:
# create data container for YCbCr data
data_YCbCr = np.zeros((N_y, N_x, 3), dtype=np.uint8)

We will asume that both the $\mathrm{RGP}$ and the $\mathrm{Y C_B C_R}$ data will be stored as 8 bit unsigned int, thus each component of the *digital* data is in the integer range:

*$\mathrm{RGP}$ colorspace:*
$$R^\prime_d G^\prime_d, B^\prime_d \in \{0,1,\dots, 254, 255\}$$

*$\mathrm{Y C_B C_R}$ colorspace:*
$$ Y^\prime, C_B, C_R \in \{0,1,\dots, 254, 255\}$$


For `jpeg` the following conversion is used (in contrast to some other conversions as e.g. for the PAL TV standard). which covers the full range of $\{0,1,\dots,255\}$.

$$ 
\left( \begin{array}{ccc} Y^\prime\\ Cb \\ Cr \end{array}\right) 
\approx 
\left( \begin{array}{ccc} 0 \\ 128 \\ 128 \end{array}\right) 
+
\left[ \begin{array}{ccc} 
0.299 & 0.587 & 0.114 \\
-0.168736 & -0.331264 & 0.5 \\
0.5 & -0.418688 & -0.081312 
\end{array}\right]
\cdot 
\left( \begin{array}{ccc} R^\prime_d \\ G^\prime_{d} \\ B^\prime_d \end{array}\right) 
$$

This conversion needs to be applied $\forall$ pixels. 

In [ ]:
# perform RGP to YCbCr conversion on all pixels
for i_y in range(N_y):
    for i_x in range(N_x):
        data_YCbCr[i_y, i_x, 0] = np.uint8(0.   + 0.299    * data_RGB[i_y, i_x, 0] + 0.587    * data_RGB[i_y, i_x, 1] + 0.114    * data_RGB[i_y, i_x, 2])
        data_YCbCr[i_y, i_x, 1] = np.uint8(128. - 0.168736 * data_RGB[i_y, i_x, 0] - 0.331264 * data_RGB[i_y, i_x, 1] + 0.5      * data_RGB[i_y, i_x, 2])
        data_YCbCr[i_y, i_x, 2] = np.uint8(128. + 0.5      * data_RGB[i_y, i_x, 0] - 0.418688 * data_RGB[i_y, i_x, 1] - 0.081312 * data_RGB[i_y, i_x, 2])

In [ ]:
# plot YCbCr output for each channel
def make_pretty():
    plt.xlabel("x-axis", fontsize=18)
    plt.xticks(fontsize=14)
    plt.ylabel("y-axis", fontsize=18)
    plt.yticks(fontsize=14)
    cb = plt.colorbar()
    for i in cb.ax.get_yticklabels():
        i.set_fontsize(14)
    plt.tight_layout()    
    plt.show()

plt.title("$\mathrm{Y^\prime}$ channel", fontsize=20)
plt.imshow(data_YCbCr[:,:,0] , cmap=plt.cm.gray)
make_pretty()

plt.title("$\mathrm{C_B}$ channel", fontsize=20)
plt.imshow(data_YCbCr[:,:,1] , cmap=plt.cm.Blues)
make_pretty()

plt.title("$\mathrm{C_R}$ channel", fontsize=20)
plt.imshow(data_YCbCr[:,:,2] , cmap=plt.cm.Reds)
make_pretty()

Please **be aware** that the color bars chosen for each channel do not represent a real perception of the $\mathrm{Y C_B C_R}$ color space. 


## Chroma subsampling

Since the human eye is far more receptive to brightness differences (luma) than color (chroma) differences, one can [subsample the chroma color channels](https://en.wikipedia.org/wiki/Chroma_subsampling) without losing relevant information for human perception. 

For `jpeg` chroma subsampling does not need to be performed necessarily. But if it is applied, the `4:2:0` subsampling is used. In order to apply this subsampling scheme, we will average the values of $2 \times 2$ pixel blocks in chroma color channels.  


In [ ]:
# Y color data stays the same:
data_Y = np.empty((N_y, N_x), dtype=np.uint8)
data_Y[:, :] = data_YCbCr[:, :, 0]

In [ ]:
# Cb and Cr color data gets subsampled
data_Cb = np.empty((N_y//2, N_x//2), dtype=np.uint8)
data_Cr = np.empty((N_y//2, N_x//2), dtype=np.uint8)

for i_y in range(0, N_y, 2):
    for i_x in range(0, N_x, 2):
        data_Cb[i_y//2, i_x//2] = np.uint8((np.uint16(data_YCbCr[i_y   , i_x   , 1]) + 
                                            np.uint16(data_YCbCr[i_y+1 , i_x   , 1]) + 
                                            np.uint16(data_YCbCr[i_y   , i_x+1 , 1]) + 
                                            np.uint16(data_YCbCr[i_y+1 , i_x+1 , 1]) ) 
                                           // 4)
        data_Cr[i_y//2, i_x//2] = np.uint8((np.uint16(data_YCbCr[i_y   , i_x   , 2]) + 
                                            np.uint16(data_YCbCr[i_y+1 , i_x   , 2]) + 
                                            np.uint16(data_YCbCr[i_y   , i_x+1 , 2]) + 
                                            np.uint16(data_YCbCr[i_y+1 , i_x+1 , 2]) ) 
                                           // 4)     

In [ ]:
# plot YCbCr output for each channel

plt.title("$\mathrm{Y^\prime}$ channel", fontsize=20)
plt.imshow(data_Y , cmap=plt.cm.gray)
make_pretty()

plt.title("$\mathrm{C_B}$ channel", fontsize=20)
plt.imshow(data_Cb , cmap=plt.cm.Blues)
make_pretty()

plt.title("$\mathrm{C_R}$ channel", fontsize=20)
plt.imshow(data_Cr , cmap=plt.cm.Reds)
make_pretty()

## Discrete cosine transform

By transforming the spatial $\mathrm{Y C_B C_R}$ data into the frequency domain, the spectral components can be determined and later be manipulated (see quantization). This transformation is called a [discrete cosine transform](https://en.wikipedia.org/wiki/Discrete_cosine_transform) (DCT). 

### Working blocks

The discrete cosine transform will be performed on $\mathrm{8 \times 8}$ pixel blocks for each of the $\mathrm{Y C_B C_R}$ channels independently. Due to down-sampling in the previous step, a $\mathrm{8 \times 8}$ pixel block in the $\mathrm{C_B}$ or $\mathrm{B_R}$ channel represents a $16 \times 16$ pixel block in the $\mathrm{Y^\prime}$ channel, or four $\mathrm{8 \times 8}$ pixel blocks. These $16 \times 16$ pixel blocks are called Minimum Coded Unit (MCU) blocks or [macroblocks](https://en.wikipedia.org/wiki/Macroblock). 

### Range modification
As a first step, the integer range used in the $\mathrm{Y C_B C_R}$ color space will be changed from 8 bit unsigned integers into ($\{0,1,\dots, 255\}$) to 8 bit (signed) integers ($\{-128, \dots, +127 \}$). This reduced the dynamic range that the discrete cosine transform needs to handle afterward. As you can see in the images above, all 3 color channels have values closely around $128$ and thus will be centered around $0$ after the range modification. 

In [ ]:
# create arrays for data with modified range
data_Y_int = np.empty((N_y, N_x), dtype=np.int8)
data_Cb_int = np.empty((N_y//2, N_x//2), dtype=np.int8)
data_Cr_int = np.empty((N_y//2, N_x//2), dtype=np.int8)


In [ ]:
# convert Y data:
for i_y in range(N_y):
    for i_x in range(N_x):
        data_Y_int[i_y, i_x] = np.int8(data_Y[i_y, i_x] - 128)

In [ ]:
# convert Cb and Cr data:
for i_y in range(N_y//2):
    for i_x in range(N_x//2):
        data_Cb_int[i_y, i_x] = np.int8(data_Cb[i_y, i_x] - 128)
        data_Cr_int[i_y, i_x] = np.int8(data_Cr[i_y, i_x] - 128)        

In [ ]:
# plot YCbCr output for each channel after range modification

plt.title("$\mathrm{Y^\prime}$ channel", fontsize=20)
plt.imshow(data_Y_int , cmap=plt.cm.gray)
make_pretty()

plt.title("$\mathrm{C_B}$ channel", fontsize=20)
plt.imshow(data_Cb_int , cmap=plt.cm.Blues)
make_pretty()

plt.title("$\mathrm{C_R}$ channel", fontsize=20)
plt.imshow(data_Cr_int , cmap=plt.cm.Reds)
make_pretty()

### Performing the discrete cosine transform on one $8 \times 8$ blocks

In the following, we will extract $8 \times 8$ blocks from each channel and perform a discrete cosine transform on them. The transformed data will be stored in $G_\mathrm{int} \in \mathbb{R^{8\times 8}}$.

The discrete cosine transform is defined as:
$$
 G[i_u,i_v] = \frac{\alpha(i_u) \cdot \alpha(i_v)}{4}
 \sum _{i_x=0}^{7}\sum _{i_y=0}^{7} g[i_x,i_y] 
 \cos \left( \frac {(2i_x+1)i_u\pi }{16} \right)
 \cos \left( \frac {(2i_y+1)i_v\pi }{16} \right)
 $$
 with
 $$
 \alpha(i) = 
 \begin{cases}
i=0 : \frac{1}{\sqrt{2}} \approx 0.707 \\
i \not = 0 : 1
\end{cases}
 $$

In [ ]:
def perform_DCT_on_block(g_int, offset_y = 0, offset_x = 0):

    # prepare array for transformed data 
    G_int = np.empty((8,8), dtype=np.float)

    # 1/sqrt(2) constant
    sqrt2_inv = 1.0 / np.sqrt(2)

    # run through all frequencies
    for i_v in range(8):
        for i_u in range(8):
            # set to zero:
            G_int[i_v, i_u] = 0.0
            # sum over all pixels
            for i_y in range(8):
                for i_x in range(8):
                    G_int[i_v, i_u] += (g_int[offset_y + i_y, offset_x + i_x] * 
                                        np.cos((2. * i_x + 1) * i_u * np.pi / 16.) * 
                                        np.cos((2. * i_y + 1) * i_v * np.pi / 16.))
            # compute factor                            
            alpha_v = 1.0
            alpha_u = 1.0
            if i_v == 0:
                alpha_v = sqrt2_inv
            if i_u == 0:
                alpha_u = sqrt2_inv
            G_int[i_v, i_u] *= 1./4. * alpha_v * alpha_u
    
    # return 8x8 transformed data
    return G_int

### Comment on the complexity of computing 

The simple discrete cosine transform has a complexity of computing of [$\mathcal{O}(n^2)$](https://en.wikipedia.org/wiki/Discrete_cosine_transform#Computation), for each $G_\mathcal{int}$, with $n$ being the block edge length. However, by applying a [divide and conquer algorithm](https://en.wikipedia.org/wiki/Divide-and-conquer_algorithm) approach, a [fast Fourier transform](https://en.wikipedia.org/wiki/Fast_Fourier_transform) (FFT) like implementation can be used. Such an algorithm is called **fast cosine transform** (FCT) and has a complexity of computing of [$\mathcal{O}(n \cdot \mathrm{log}(n))$](https://en.wikipedia.org/wiki/Discrete_cosine_transform#Computation). Alternatively one can use the FFT algorithm, which has also the complexity of $\mathcal{O}(n \cdot \mathrm{log}(n))$ but is commonly available as an optimized library for various hardware. However, using the FFT algorithm instead leads to an additional data conversion of complexity $\mathcal{O}(n)$. 

Since the difference in computational complexity is significant, an FCT should always be preferred over a DCT.  

### transforming one $8 \times 8$ block of the image
In the following, we select a random $8 \times 8$ block of the image $\mathrm{Y^\prime}$ channel and study the transformed data. 

In [ ]:
# compute DCT of a 8x8 block of Y channel:

# block offset
offset_y = 55 * 8
offset_x = 26 * 8

# perform DCT
G_int_0_0 = perform_DCT_on_block(data_Y_int, offset_y, offset_x)

In [ ]:
# plot DCT result
plt.title("$\mathrm{DCT(Y^\prime)}$ channel", fontsize=20)
plt.imshow(G_int_0_0 , norm=LogNorm())
make_pretty()

As can be seen above, the $(0,0)$ component has the strongest contribution. This is the so-called **DC** ("direct current" - originating from electric currents) term. It describes the constant background of this $8 \times 8$ pixel block. All other components are called **AC** ("alternating current") and have some form of variation along either $x$ or $y$ axis. 


## Quantization

The fact that small scale variations on an $8 \times 8$ pixel area are not as important for human perception, allows us to filter out higher frequency components $i_u > 0$ and $i_v > 0$ in the next step. This is done by the so-called "quantization" step. It convert the DCT data $\mathbb{R}^{8 \times 8}$ to 8-bit (signed) integer values $\mathbb{Z}_{8} = \{ -128, \dots, 127\}$. This is done by dividing the data by a frequency depend [quantization matrix](https://en.wikipedia.org/wiki/Quantization_(image_processing)#Quantization_matrices) $Q_p[i_v, i_u]$. 
$$
B[i_v, i_u] = \mathrm{round} \left( \frac{G[i_v, i_u]}{Q[i_v,i_u]} \right) 
\quad \forall i_u, i_v \in \{ 0, 1, \dots ,7\}
$$

with $Q_p$ being a quality depending $8 \times 8$ matrix $\in \mathbb{N}_8^{8 \times 8}$. For example the "degree 50", aka "high quality" matrix has the form:

$$
 Q_{50}= \left[ \begin{array}{cccccccc}
 16 & 11 & 10 & 16 &  24 &  40 &  51 &  61 \\
 12 & 12 & 14 & 19 &  26 &  58 &  60 &  55 \\
 14 & 13 & 16 & 24 &  40 &  57 &  69 &  56 \\
 14 & 17 & 22 & 29 &  51 &  87 &  80 &  62 \\
 18 & 22 & 37 & 56 &  68 & 109 & 103 &  77 \\
 24 & 35 & 55 & 64 &  81 & 104 & 113 &  92 \\
 49 & 64 & 78 & 87 & 103 & 121 & 120 & 101 \\
 72 & 92 & 95 & 98 & 112 & 100 & 103 &  99
 \end{array} \right]
$$



In [ ]:
# define Q matrix
Q_50 = np.array([
                 [ 16 , 11 , 10 , 16 ,  24 ,  40 ,  51 ,  61 ],
                 [ 12 , 12 , 14 , 19 ,  26 ,  58 ,  60 ,  55 ],
                 [ 14 , 13 , 16 , 24 ,  40 ,  57 ,  69 ,  56 ],
                 [ 14 , 17 , 22 , 29 ,  51 ,  87 ,  80 ,  62 ],
                 [ 18 , 22 , 37 , 56 ,  68 , 109 , 103 ,  77 ],
                 [ 24 , 35 , 55 , 64 ,  81 , 104 , 113 ,  92 ],
                 [ 49 , 64 , 78 , 87 , 103 , 121 , 120 , 101 ],
                 [ 72 , 92 , 95 , 98 , 112 , 100 , 103 ,  99 ]      
                ], dtype=np.uint8)

In [ ]:
def perform_quantization_on_block(G_int, Q):
    B_int = np.empty((8,8), dtype=np.int8)
    for i_v in range(8):
        for i_u in range(8):
            B_int[i_v, i_u] = np.int8( np.round( G_int[i_v, i_u] / Q[i_v, i_u] ) )
    return B_int

In [ ]:
# perform quantization
B_int = perform_quantization_on_block(G_int_0_0, Q_50)

In [ ]:
# plot DCT result
plt.title("$\mathrm{DCT(Y^\prime)}$ channel", fontsize=20)
plt.imshow(B_int, vmin=-1.0*np.amax(np.abs(B_int)), vmax=+1.0*np.amax(np.abs(B_int)), cmap=plt.cm.RdBu_r )
make_pretty()

print(B_int)

The quantization leads to an array, with most components farther away from the DC component at $(0,0)$ being zero. Only the AC terms $(i_v, i_u) \not = (0,0)$ close to the DC term are non-zero. This means that this specific $8 \times 8$ pixel block does not contain higher frequency fluctuations that alter at a pixel distance. 

## Entropy encoding
Theoretically all $256^{8 \cdot 8} = 2^{512} \approx 10^{170}$ possible $B$ arrays could occur. However, images tend to show structures over many pixels, thus $8 \times 8$ pixel blocks commonly do not contain a lot of variation. 
Thus in Fourier space, the DC tame and only small frequency AC term will tend to be non-zero. This allows for an [entropy encoding](https://en.wikipedia.org/wiki/Entropy_encoding) that maps the fixed size $8 \times 8$ input array $B$ to a variable-length output codeword, with the most likely input arrays $B$ resulting in the shortest code words. 

The `jpeg` standard allows performing entropy encoding via [Huffman coding](https://en.wikipedia.org/wiki/Huffman_coding) or [arithmetic coding](https://en.wikipedia.org/wiki/Arithmetic_coding), with the first being the most common due to historic patent regulations. Here we will focus solely on Huffman coding. 


### Huffman coding
The Huffman coding used in `jpeg` applies a zig-zag reordering of the spectral data in $B$ in such a manner, that the reordered data will go from low to high frequency. The order of traversing $B$ is given with C-like indices. 

$$
\left[ \begin{array}{cccccccc}
  0 &  1 &  5 &  6 & 14 & 15 & 27 & 28 \\
  2 &  4 &  7 & 13 & 16 & 26 & 29 & 42 \\
  3 &  8 & 12 & 17 & 25 & 30 & 41 & 43 \\
  9 & 11 & 18 & 24 & 31 & 40 & 44 & 53 \\
 10 & 19 & 23 & 32 & 39 & 45 & 52 & 54 \\
 20 & 22 & 33 & 38 & 46 & 51 & 55 & 60 \\
 21 & 34 & 37 & 47 & 50 & 56 & 59 & 61 \\
 35 & 36 & 48 & 49 & 57 & 58 & 62 & 63
 \end{array} \right]
$$

In [ ]:
# implement zig-zag access pattern

zig_zag_index = []
left_right = False
start = 0

for length in range(15):
    entry = []
    if length >=8:
        start += 1
    for index_1 in range(start, length+1-start):
        index_2 = length - index_1
        if left_right:
            entry.append((index_1, index_2))
        else:
            entry.append((index_2, index_1))
        
    zig_zag_index.append(entry)
    left_right = not left_right
    

In [ ]:
# print index-pairs for each diagonal 
zig_zag_index

In [ ]:
# flatten list of diagnoals 
zig_zag_flat = [index_pair for diagonal in zig_zag_index for index_pair in diagonal]

In [ ]:
# validate that correct access pattern was generated
access_pattern = np.empty((8,8), dtype=np.uint8)
for i in range(64):
    access_pattern[zig_zag_flat[i]] = i
    
print("access pattern:\n", access_pattern)

In the `jpeg` encoding, the DC term will be stored separately from the AC terms. The AC terms will be compressed by applying a [run-length_encoding](https://en.wikipedia.org/wiki/Run-length_encoding) on the zero values. 

The AC data is stored in 1-2 8 bit entries. All entries with data will be 8 bit long, only the last one will be 4 bit long. 

- The first 4 bits are a 4-bit integer giving the number of zero amplitude values passed by the zig-zag access pattern before. 
- The second 4-bit state is the number of 1/True bits in the amplitude as a 4-bit integer. Both 4-bit values are called "symbol 1"
- The following 8 bits store the (non-zero) amplitude. These 8 bits are colled "symbol 2".

If only trailing zeros are left, 8 bits equal to the integer `0`  representation are stored. 

In [ ]:
def apply_Huffman_encoding(B_int, verbose=False):

    output = []

    runlength = 0
    overflow_counter = 0
    for index_pair_id in range(1, len(zig_zag_flat)):
        index_pair = zig_zag_flat[index_pair_id]
        #print(index_pair)
        amplitude = B_int[index_pair]
        if B_int[index_pair] == 0:
            runlength += 1
            if runlength == 15:
                overflow_counter += 1
                runlength = 0        
        else:
            #print(runlength)
            size = 0
            number = np.abs(amplitude)
            while (number): 
                size += number & 1
                number >>= 1
            for i in range(overflow_counter):
                output.append(np.int8(runlength)<<4) # symbol 1
                output.append(np.int8(0)) # symbol 2
                if verbose:
                    print("({:d},{:d})({:d});".format(runlength, 0, 0))

            output.append( (np.int8(runlength)<<4) + np.int8(size) )# symbol 1
            output.append(np.int8(amplitude)) # symbol 2
            if verbose:
                print("({:d},{:d})({:d});".format(runlength, size, amplitude))
            runlength = 0

    if verbose:
        print("({:d},{:d});".format(0, 0))
    output.append(np.int8(0))
    
    return np.int8(B_int[0,0]), np.array(output, dtype=np.int8)


In [ ]:
dc_encoded, ac_encoded = apply_Huffman_encoding(B_int, verbose=True)

print(dc_encoded, ac_encoded)

## Apply DCT, quantization, and Huffman encoding to all blocks:

### $\mathrm{Y^\prime}$ channel:

In [ ]:
# determine number of blocks:
N_x_blocks = N_x //8
N_y_blocks = N_y //8

In [ ]:
Y_dc = np.empty((N_y_blocks, N_x_blocks), dtype=np.int8)
Y_ac = []

for i_y_b in range(N_y_blocks):
    for i_x_b in range(N_x_blocks):
        # block offset
        offset_y = i_y_b * 8
        offset_x = i_x_b * 8

        # perform DCT
        G = perform_DCT_on_block(data_Y_int, offset_y, offset_x)
        
        # perform quantization
        B = perform_quantization_on_block(G, Q_50)
        
        # perform entropy encoding
        dc_encoded, ac_encoded = apply_Huffman_encoding(B, verbose=False)
        
        # store data
        Y_dc[i_y_b, i_x_b] = dc_encoded
        Y_ac.append(ac_encoded)
        
Y_ac = np.array([symbol for block in Y_ac for symbol in block], dtype=np.int8)


In [ ]:
print("compressed to {:.1f} %".format((len(Y_ac) + len(Y_dc))/(N_y * N_x) * 100.))

In [ ]:
# plot Y dc values
plt.title("$\mathrm{dc(Y^\prime)}$ channel", fontsize=20)
plt.imshow(Y_dc, cmap=plt.cm.gray)
make_pretty()

### $\mathrm{C_B}$ channel

In [ ]:
# determine number of blocks:
N_x_blocks_c = N_x //8 //2
N_y_blocks_c = N_y //8 //2

In [ ]:
Cb_dc = np.empty((N_y_blocks_c, N_x_blocks_c), dtype=np.int8)
Cb_ac = []

for i_y_b in range(N_y_blocks_c):
    for i_x_b in range(N_x_blocks_c):
        # block offset
        offset_y = i_y_b * 8
        offset_x = i_x_b * 8

        # perform DCT
        G = perform_DCT_on_block(data_Cb_int, offset_y, offset_x)
        
        # perform quantization
        B = perform_quantization_on_block(G, Q_50)
        
        # perform entropy encoding
        dc_encoded, ac_encoded = apply_Huffman_encoding(B, verbose=False)
        
        # store data
        Cb_dc[i_y_b, i_x_b] = dc_encoded
        Cb_ac.append(ac_encoded)
        
Cb_ac = np.array([symbol for block in Cb_ac for symbol in block], dtype=np.int8)


In [ ]:
print("compressed to {:.1f} %".format((len(Cb_ac) + len(Cb_dc))/(N_y * N_x) * 100.))

In [ ]:
# plot Cb dc values
plt.title("$\mathrm{dc(C_B)}$ channel", fontsize=20)
plt.imshow(Cb_dc, cmap=plt.cm.Blues)
make_pretty()

### $\mathrm{C_R}$ channel

In [ ]:
Cr_dc = np.empty((N_y_blocks_c, N_x_blocks_c), dtype=np.int8)
Cr_ac = []

for i_y_b in range(N_y_blocks_c):
    for i_x_b in range(N_x_blocks_c):
        # block offset
        offset_y = i_y_b * 8
        offset_x = i_x_b * 8

        # perform DCT
        G = perform_DCT_on_block(data_Cr_int, offset_y, offset_x)
        
        # perform quantization
        B = perform_quantization_on_block(G, Q_50)
        
        # perform entropy encoding
        dc_encoded, ac_encoded = apply_Huffman_encoding(B, verbose=False)
        
        # store data
        Cr_dc[i_y_b, i_x_b] = dc_encoded
        Cr_ac.append(ac_encoded)
        
Cr_ac = np.array([symbol for block in Cr_ac for symbol in block], dtype=np.int8)


In [ ]:
print("compressed to {:.1f} %".format((len(Cr_ac) + len(Cr_dc))/(N_y * N_x) * 100.))

In [ ]:
# plot Cr dc values
plt.title("$\mathrm{dc(C_R)}$ channel", fontsize=20)
plt.imshow(Cr_dc, cmap=plt.cm.Reds)
make_pretty()

## Compression of `jpeg`

For the above case, the image compresses dramatically since most blocks have only the same color pixels thus causing all AC terms to become zero. 

In [ ]:
print("compressed to {:.1f} %".format((len(Y_ac) + len(Y_dc) + 
                                       len(Cb_ac) + len(Cb_dc) + 
                                       len(Cr_ac) + len(Cr_dc))/(3 * N_y * N_x) * 100.))

## Write `jpeg` file

Writing the actual `jpeg` files requires various binary outputs. This will perhaps be added later since it does not help much in understanding the main algorithm of encoding `jpegs`. 